In [17]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV

In [22]:
data = pd.read_csv('data/train.csv')
data_test = pd.read_csv('data/test.csv')

In [53]:
data_test

,ID,Age,AI_Interaction_Level,Satisfaction_with_AI_Services,AI_Personalization_Effectiveness,AI_Response_Time,Overall_Usage_Frequency,Customer_Service_Interactions,Change_in_Usage_Patterns,Customer_Churn
0,1,59,1,2,3,1,11,5,-1,1
1,2,49,3,4,3,2,2,0,2,1
2,3,35,3,7,6,2,22,4,0,1
3,4,63,2,8,8,1,25,4,-1,1
4,5,28,4,5,5,2,14,1,1,1
...,...,...,...,...,...,...,...,...,...,...
1245,1246,42,3,1,1,3,12,5,-1,1
1246,1247,63,2,10,9,2,10,6,0,0
1247,1248,24,4,9,10,2,2,4,1,0
1248,1249,46,3,7,7,3,12,1,0,0


In [89]:
X = data.iloc[0:, :-1].values
y = data.iloc[0:,  -1].values

X_test = data_test.iloc[0:, :-1].values
y_test = data_test.iloc[0:,  -1].values

X = np.delete(X, [0], axis=1)
X_test = np.delete(X_test, [0], axis=1)

for i in [0]:
    X[:,i] = (X[:,i] - np.mean(X[:, i]) )/ np.std(X[:,i]) 
    X_test[:,i] = (X_test[:,i] - np.mean(X_test[:, i]) )/ np.std(X_test[:,i]) 
    
# X[:,0] = (X[:,0] - np.mean(X[:, 0]) )/ np.std(X[:,0]) 
# X_test[:,0] = (X_test[:,0] - np.mean(X_test[:, 0]) )/ np.std(X_test[:,0]) 

X.shape, y.shape, X_test.shape, y_test.shape
X

array([[ 1,  2,  7, ..., 18,  5,  2],
       [ 0,  2,  6, ...,  3,  2,  0],
       [ 0,  2,  3, ...,  8,  2,  1],
       ...,
       [ 0,  3,  6, ..., 15,  8,  2],
       [ 0,  3,  7, ..., 21,  6,  0],
       [ 0,  2,  6, ..., 24,  6, -2]], dtype=int64)

In [90]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
# from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Define the base classifiers
rf_clf = RandomForestClassifier(n_estimators=150, random_state=42)
knn_clf = KNeighborsClassifier(n_neighbors=3)
svm_clf = SVC(kernel='rbf', probability=True, random_state=42)
# xgb_clf = XGBClassifier(learning_rate=0.1, n_estimators=100, random_state=42)
gbm_model = GradientBoostingClassifier(random_state=42, n_estimators=300, learning_rate= 0.03, max_depth = 8)

# Create the ensemble model using a VotingClassifier
ensemble_clf = VotingClassifier(estimators=[
    ('random_forest', rf_clf),
    ('knn', knn_clf),
    ('svm', svm_clf),
    # ('xgboost', xgb_clf)
    ('gbm', gbm_model)
], voting='soft')  # 'soft' voting for probability prediction

# Train the ensemble model
ensemble_clf.fit(X, y)

# Make predictions
y_pred = ensemble_clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Ensemble Model Accuracy:", accuracy)

Ensemble Model Accuracy: 0.7


In [93]:
subm1 = y_pred

ids = np.arange(1, len(subm1) + 1)[:, np.newaxis]  # Create a column vector of sequential IDs

# Add the ID column to the vector
vector_with_ids = np.hstack((ids, subm1[:, np.newaxis]))
vector_with_ids

array([[   1,    0],
       [   2,    0],
       [   3,    0],
       ...,
       [1248,    0],
       [1249,    1],
       [1250,    0]], dtype=int64)

In [94]:



# Define the file path
file_path = 'subm2.csv'

# Export the NumPy array to a CSV file
np.savetxt(file_path, vector_with_ids, delimiter=',', fmt='%d')

In [74]:
X_test.shape, y_pred.shape

((1250, 8), (1250,))

In [7]:
X = data.iloc[:, :-1].values
y = data.iloc[:,  -1].values


In [8]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)
rf_train_preds = rf_model.predict(X_train)
rf_valid_pred  = rf_model.predict(X_valid)
rf_train_error = 1 - accuracy_score(y_train, rf_train_preds)
rf_valid_error = 1 - accuracy_score(y_valid, rf_valid_pred)
print("Random Forest Training Error:", rf_train_error)
print("Random Forest Validation Error:", rf_valid_error)


Random Forest Training Error: 0.0
Random Forest Validation Error: 0.42100000000000004


In [16]:
# Gradient Boosting Machine
gbm_model = GradientBoostingClassifier(random_state=42, n_estimators=200, learning_rate= 0.01, max_depth = 8)
gbm_model.fit(X_train, y_train)

# Training error
gbm_train_preds = gbm_model.predict(X_train)
gbm_train_error = 1 - accuracy_score(y_train, gbm_train_preds)
print("Gradient Boosting Machine Training Error:", gbm_train_error)

# Validation error
gbm_valid_preds = gbm_model.predict(X_valid)
gbm_valid_error = 1 - accuracy_score(y_valid, gbm_valid_preds)
print("Gradient Boosting Machine Validation Error:", gbm_valid_error)

Gradient Boosting Machine Training Error: 0.13024999999999998
Gradient Boosting Machine Validation Error: 0.406


In [19]:
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7]
}

grid_search = GridSearchCV(gbm_model, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X, y)


GridSearchCV(cv=5,
             estimator=GradientBoostingClassifier(learning_rate=0.01,
                                                  max_depth=8, n_estimators=200,
                                                  random_state=42),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 5, 7],
                         'n_estimators': [50, 100, 200]},
             scoring='accuracy')

In [22]:
print("Best Parameters:", grid_search.best_params_)
print("Validation Error (accuracy):", 1 - grid_search.best_score_)

Best Parameters: {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 50}
Validation Error (accuracy): 0.4454
